# Aggregations
Data aggregation is the process where raw data is gathered and expressed in a summary form for statistical analysis.

For example, raw data can be aggregated over a given time period to provide analysis statistics such as average, minimum, maximum, sum and count. After the data is aggregated and written to a view or report, you can analyze the aggregated data to gain insights about particular resources or resource grous. 

There are two types of data aggregation: 
* Time aggregation (all data points for a single resource over a specified time period)
* Spatial aggregation (all data points for a group of resources over a specified time period).

In [2]:
import numpy as np
import pandas as pd


In [3]:
## Built-in aggregation mathods an Pandas
numbers = pd.Series(np.random.randint(low=1, high=100, size=5))
numbers

0    88
1    53
2    54
3    44
4     4
dtype: int32

In [4]:
## Some of Pandas' aggregation methods are:
print(f"{numbers.min() = }")
print(f"{numbers.max() = }")
print(f"{numbers.sum() = }")
print(f"{numbers.mean() = }")
print(f"{numbers.median() = }")
print(f"{numbers.count() = }")
print(f"{numbers.size() = }")
print(f"{numbers.mode()[0] = }")

numbers.min() = 4
numbers.max() = 88
numbers.sum() = 243
numbers.mean() = 48.6
numbers.median() = 53.0
numbers.count() = 5


TypeError: 'int' object is not callable

## When run on a dataframe (multiple Series) they return a single value for each Series, forming a new Series

In [ ]:
numbers_df = pd.DataFrame(np.random.randint(low=1,high=100,size=[5,5]))
numbers_df

,0,1,2,3,4
0,86,96,33,56,73
1,39,6,8,44,51
2,13,7,55,59,43
3,52,52,80,98,79
4,21,79,21,25,10


In [ ]:
numbers_df.min(axis="columns") # Default "index", checks all rows for min, now all columns
numbers_df.min(axis="columns").min() # Gets min value of entire table


6

## Working with real data

In [6]:
autos = pd.read_json("../Data/autos.json")
autos.tail(3)

,aspiration,body-style,bore,city-mpg,compression-ratio,curb-weight,drive-wheels,engine-location,engine-size,engine-type,...,make,normalized-losses,num-of-cylinders,num-of-doors,peak-rpm,price,stroke,symboling,wheel-base,width
202,std,sedan,3.58,18,8.8,3012,rwd,front,173,ohcv,...,volvo,95.0,six,four,5500.0,21485.0,2.87,-1,109.1,68.9
203,turbo,sedan,3.01,26,23.0,3217,rwd,front,145,ohc,...,volvo,95.0,six,four,4800.0,22470.0,3.40,-1,109.1,68.9
204,turbo,sedan,3.78,19,9.5,3062,rwd,front,141,ohc,...,volvo,95.0,four,four,5400.0,22625.0,3.15,-1,109.1,68.9


In [ ]:
# To get average price
autos["price"].mean()

13207.129353233831

In [ ]:
autos[["length", "width", "height"]].head(3)

,length,width,height
0,168.8,64.1,48.8
1,168.8,64.1,48.8
2,171.2,65.5,52.4


In [ ]:
# Running mean() on multiple columns (DataFrame) returns a Series of means.

autos[["length", "width", "height"]].head(3).mean()

length    169.600000
width      64.566667
height     50.000000
dtype: float64

In [ ]:
# Only for Volvo cars
autos[autos["make"]=="volvo"][["length", "width", "height"]].head(3).mean()

# Another method
autos.query("make == 'volvo'")[["length", "width", "height"]].head(3).mean()

# autos[["length", "width", "height"]].head(3).mean()


length    188.800000
width      67.200000
height     56.633333
dtype: float64

## Multiple aggregation

In [ ]:
autos[["length", "width", "height"]].agg(["min", "mean", "max"])

# Getting min mean and max for all numerical columns
autos[[col for col in autos.columns if autos[col].dtype in ["int64", "float64"]]].agg(["min", "mean", "max"])

# Same as describe
autos.describe()

,bore,city-mpg,compression-ratio,curb-weight,engine-size,height,highway-mpg,horsepower,length,normalized-losses,peak-rpm,price,stroke,symboling,wheel-base,width
min,2.540000,13.000000,7.000000,1488.000000,61.000000,47.800000,16.00000,48.000000,141.100000,65.0,4150.000000,5118.000000,2.070000,-2.000000,86.600000,60.300000
mean,3.329751,25.219512,10.142537,2555.565854,126.907317,53.724878,30.75122,104.256158,174.049268,122.0,5125.369458,13207.129353,3.255423,0.834146,98.756585,65.907805
max,3.940000,49.000000,23.000000,4066.000000,326.000000,59.800000,54.00000,288.000000,208.100000,256.0,6600.000000,45400.000000,4.170000,3.000000,120.900000,72.300000


### Split-Apply-Combine
* Select a feature to use as key
* Split the dataset into group for each unique key value
* Apply aggregation to each group
* Combine aggregated data into a new dataset

![image](../assets/split-apply-combine.svg)

### Group by
Use Pandas .groupby() method to select a key and split into groups.

This creates a new DataFrameGroupBy object containing the grouped DataFrames.

In [ ]:
makes = autos.groupby("make")
type(makes)

pandas.core.groupby.generic.DataFrameGroupBy

In [ ]:
len(makes)

22

In [ ]:
autos["make"].nunique() ## Check that there are 22 makes
autos["make"].value_counts() # Count all occurences of each

make
toyota           32
nissan           18
mazda            17
mitsubishi       13
honda            13
volkswagen       12
subaru           12
peugot           11
volvo            11
dodge             9
mercedes-benz     8
bmw               8
audi              7
plymouth          7
saab              6
porsche           5
isuzu             4
jaguar            3
chevrolet         3
alfa-romero       3
renault           2
mercury           1
Name: count, dtype: int64

In [ ]:
makes.groups
autos.loc[makes.groups["jaguar"]]
# Same as above, easier
makes.get_group("jaguar")

,aspiration,body-style,bore,city-mpg,compression-ratio,curb-weight,drive-wheels,engine-location,engine-size,engine-type,...,make,normalized-losses,num-of-cylinders,num-of-doors,peak-rpm,price,stroke,symboling,wheel-base,width
47,std,sedan,3.63,15,8.1,4066,rwd,front,258,dohc,...,jaguar,145.0,six,four,4750.0,32250.0,4.17,0,113.0,69.6
48,std,sedan,3.63,15,8.1,4066,rwd,front,258,dohc,...,jaguar,NaN,six,four,4750.0,35550.0,4.17,0,113.0,69.6
49,std,sedan,3.54,13,11.5,3950,rwd,front,326,ohcv,...,jaguar,NaN,twelve,two,5000.0,36000.0,2.76,0,102.0,70.6


### Apply and combine
It's possible to access a single group as above.
However, most of the time we rather apply aggregation functions to each group individually and comvine the result into a new dataset.

In [ ]:
makes.count()

,aspiration,body-style,bore,city-mpg,compression-ratio,curb-weight,drive-wheels,engine-location,engine-size,engine-type,...,length,normalized-losses,num-of-cylinders,num-of-doors,peak-rpm,price,stroke,symboling,wheel-base,width
make,,,,,,,,,,,,,,,,,,,,,
alfa-romero,3,3,3,3,3,3,3,3,3,3,...,3,0,3,3,3,3,3,3,3,3
audi,7,7,7,7,7,7,7,7,7,7,...,7,4,7,7,7,6,7,7,7,7
bmw,8,8,8,8,8,8,8,8,8,8,...,8,4,8,8,8,8,8,8,8,8
chevrolet,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
dodge,9,9,9,9,9,9,9,9,9,9,...,9,9,9,8,9,9,9,9,9,9
honda,13,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13
isuzu,4,4,4,4,4,4,4,4,4,4,...,4,0,4,4,4,2,4,4,4,4
jaguar,3,3,3,3,3,3,3,3,3,3,...,3,1,3,3,3,3,3,3,3,3
mazda,17,17,13,17,17,17,17,17,17,17,...,17,15,17,16,17,17,13,17,17,17


In [ ]:
makes[["length", "width", "height"]].mean()

,length,width,height
make,,,
alfa-romero,169.600000,64.566667,50.000000
audi,183.828571,68.714286,54.428571
bmw,184.500000,66.475000,54.825000
chevrolet,151.933333,62.500000,52.400000
dodge,160.988889,64.166667,51.644444
honda,160.769231,64.384615,53.238462
isuzu,163.775000,63.550000,52.225000
jaguar,196.966667,69.933333,51.133333
mazda,170.805882,65.588235,53.358824


### SeriesGroupBy
Indexing a DataFrameGroupBy object with a single column will return a SeriesGroupBy object.

In [ ]:
sgb = makes["price"]
type(sgb)
sgb.groups
sgb.get_group("jaguar")
sgb.mean().head(3)

make
alfa-romero    15498.333333
audi           17859.166667
bmw            26118.750000
Name: price, dtype: float64

In [ ]:
# Splitting over several lines 

(
autos.groupby("make")
    ["price"]
    .mean()
)

# But it's normal to have everything on one line in these cases

### Multiple aggregations on SeriesGroupBy
Use pandas .agg() method on SeriesGroupBy to do multiple aggregation on a single feature.

In [ ]:
sgb.min()
sgb.agg(["min", "mean", "max"])

# Neat way to get overview
autos.groupby("make")["price"].describe().transpose()

make,alfa-romero,audi,bmw,chevrolet,dodge,honda,isuzu,jaguar,mazda,mercedes-benz,...,nissan,peugot,plymouth,porsche,renault,saab,subaru,toyota,volkswagen,volvo
count,3.000000,6.000000,8.000000,3.000000,9.000000,13.000000,2.000000,3.000000,17.000000,8.000000,...,18.000000,11.000000,7.000000,4.000000,2.000000,6.000000,12.000000,32.000000,12.000000,11.000000
mean,15498.333333,17859.166667,26118.750000,6007.000000,7875.444444,8184.692308,8916.500000,34600.000000,10652.882353,33647.000000,...,10415.666667,15489.090909,7963.428571,31400.500000,9595.000000,15223.333333,8541.250000,9885.812500,10077.500000,18063.181818
std,1734.937559,3452.379493,9263.832033,754.421633,2213.386044,2061.672112,3014.396208,2047.559523,3975.682094,6789.560306,...,4477.394200,2246.749673,2395.544257,6528.784343,424.264069,2860.794761,1940.191468,3204.982114,2178.549872,3314.650263
min,13495.000000,13950.000000,16430.000000,5151.000000,5572.000000,5399.000000,6785.000000,32250.000000,5195.000000,25552.000000,...,5499.000000,11900.000000,5572.000000,22018.000000,9295.000000,11850.000000,5118.000000,5348.000000,7775.000000,12940.000000
25%,14997.500000,15800.000000,19958.750000,5723.000000,6377.000000,6855.000000,7850.750000,33900.000000,7395.000000,28230.000000,...,7311.500000,13530.000000,6460.500000,29900.500000,9445.000000,12887.500000,7378.750000,7870.500000,8145.000000,16250.000000
50%,16500.000000,17580.000000,22835.000000,6295.000000,7609.000000,7295.000000,8916.500000,35550.000000,10595.000000,32892.000000,...,8124.000000,16630.000000,7609.000000,33278.000000,9595.000000,15275.000000,7894.000000,9103.000000,9737.500000,18420.000000
75%,16500.000000,18617.500000,32290.000000,6435.000000,8558.000000,9095.000000,9982.250000,35775.000000,11845.000000,36532.000000,...,13499.000000,16987.500000,8439.000000,34778.000000,9745.000000,17490.000000,10019.500000,10973.250000,11768.750000,20265.000000
max,16500.000000,23875.000000,41315.000000,6575.000000,12964.000000,12945.000000,11048.000000,36000.000000,18344.000000,45400.000000,...,19699.000000,18150.000000,12764.000000,37028.000000,9895.000000,18620.000000,11694.000000,17669.000000,13845.000000,22625.000000


### Multiple aggregation on DataFrameGroupBy
Using pandas .agg() method on a DataFrameGroupBy to do multiple aggregation on multiple features will return a multi-index column datagrame.

In [ ]:
makes[["length", "width", "height"]].agg(["min","mean","max"]).head(3)

length                    width                  height  \
               min        mean    max   min       mean   max    min   
make                                                                  
alfa-romero  168.8  169.600000  171.2  64.1  64.566667  65.5   48.8   
audi         176.6  183.828571  192.7  66.2  68.714286  71.4   52.0   
bmw          176.8  184.500000  197.0  64.8  66.475000  70.9   53.7   

                              
                  mean   max  
make                          
alfa-romero  50.000000  52.4  
audi         54.428571  55.9  
bmw          54.825000  56.3

### Custom columns aggregation

In [ ]:
autos.groupby("make").agg({"price": "mean", "horsepower": "max"}).head(3)

,price,horsepower
make,,
alfa-romero,15498.333333,154.0
audi,17859.166667,160.0
bmw,26118.750000,182.0


In [31]:
# Better for presentation, since you see what the columns represent
def list_unique(x):
    return ", ".join(sorted(x.apply(str).unique()))

autos.groupby("make").agg(
    averagage_price = pd.NamedAgg(column="price", aggfunc="mean"),
    min_horsepower = pd.NamedAgg(column="horsepower", aggfunc="min"),
    max_horsepower = pd.NamedAgg(column="horsepower", aggfunc="max"),
    mean_horsepower = pd.NamedAgg(column="horsepower", aggfunc=np.mean),
    body_styles = pd.NamedAgg(column="body-style", aggfunc=list_unique)
).sort_values(by="averagage_price", ascending=False)


C:\Users\Mikae\AppData\Local\Temp\ipykernel_4480\4229595913.py:5: FutureWarning: The provided callable <function mean at 0x000001E1775E8EA0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  plot_autos = autos.groupby("make").agg(
C:\Users\Mikae\AppData\Local\Temp\ipykernel_4480\4229595913.py:13: FutureWarning: The provided callable <function mean at 0x000001E1775E8EA0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  autos.groupby("make").agg(


,averagage_price,min_horsepower,max_horsepower,mean_horsepower,body_styles
make,,,,,
jaguar,34600.000000,176.0,262.0,204.666667,sedan
mercedes-benz,33647.000000,123.0,184.0,146.250000,"convertible, hardtop, sedan, wagon"
porsche,31400.500000,143.0,288.0,210.400000,"convertible, hardtop, hatchback"
bmw,26118.750000,101.0,182.0,138.875000,sedan
volvo,18063.181818,106.0,162.0,128.000000,"sedan, wagon"
audi,17859.166667,102.0,160.0,121.000000,"hatchback, sedan, wagon"
mercury,16503.000000,175.0,175.0,175.000000,hatchback
alfa-romero,15498.333333,111.0,154.0,125.333333,"convertible, hatchback"
peugot,15489.090909,95.0,142.0,99.818182,"sedan, wagon"


In [25]:
def my_sorting_function(person):
    return person["Firstname"] # On what should the sorting be performed? Here the first name

my_list = [
    {"Firstname": "Fredrik", "Lastname": "Johansson", "Age": 42},
    {"Firstname": "Anders", "Lastname": "Svensson", "Age": 22},
    {"Firstname": "Maria", "Lastname": "Karlsson", "Age": 27},
    {"Firstname": "Anna", "Lastname": "Bengtsson", "Age": 39},
]
# my_list.sort()
#alternatively
sorted_list = sorted(my_list, key=my_sorting_function)
sorted_list = sorted(my_list, key=lambda person: person["Firstname"]) # Using lambda instead of separate function
sorted_list

[{'Firstname': 'Anders', 'Lastname': 'Svensson', 'Age': 22},
 {'Firstname': 'Anna', 'Lastname': 'Bengtsson', 'Age': 39},
 {'Firstname': 'Fredrik', 'Lastname': 'Johansson', 'Age': 42},
 {'Firstname': 'Maria', 'Lastname': 'Karlsson', 'Age': 27}]

In [30]:
def my_func():
    return "Hello"

# Equivalent to the above
myfunc = lambda: "Hello"

print(myfunc())

Hello
